In [1]:
# %load ../import.py
# import quandl
# quandl.ApiConfig.api_key = 'MBMzvkxtv63KjFEV-tL6'
# from quandl.errors.quandl_error import NotFoundError

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn import linear_model

import ipywidgets as widgets 
from ipywidgets import  interact, interactive, fixed, interact_manual
from IPython.display import display


# From tensorflow includes
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt

from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

ImportError: Traceback (most recent call last):
  File "/home/boo/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/boo/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/boo/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/boo/anaconda3/envs/tf_gpu/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/boo/anaconda3/envs/tf_gpu/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libnvidia-fatbinaryloader.so.384.130: cannot open shared object file: No

In [ ]:
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows =10
pd.options.display.float_format = '{:.1f}'.format


In [ ]:
cal_data=pd.read_csv("https://storage.googleapis.com/mledu-datasets/california_housing_train.csv", sep=",")

# cal_data=cal_data.reindex(np.random.permutation(cal_data.index))
cal_data.describe()

In [ ]:
# Output Dataset with this 

def preprocess_features(df):
    
    """
    Parameter:
        Dataframe 
    Return:
        Dataframe: with features only
    """
    
    selected_features = df[
            [
             'longitude',
             'latitude',
             'housing_median_age',
             'total_rooms',
             'total_bedrooms',
             'population',
             'households',
             'median_income',
            ]]
    
    processed_features = selected_features.copy()
    processed_features["rooms_per_person"] = (
        df["total_rooms"]/df["population"]
    )
        
    return processed_features

def preprocess_targets(df):
    """
    Parameter:
        Dataframe:
    Return:
        Datafreme with target values only
    """
    
    output_targets=pd.DataFrame()
    output_targets["median_house_value"] = (
        df["median_house_value"]/1000.0)
    
    return output_targets

# Output Feature Column

def construct_feature_columns(input_features):
    return set([
        tf.feature_column.numeric_column(my_feature)
        for my_feature in input_features 
        ])


In [ ]:
training_examples   = preprocess_features(cal_data.head(12000))
training_targets    = preprocess_targets( cal_data.head(12000))
validation_examples = preprocess_features(cal_data.tail(5000))
validation_targets  = preprocess_targets( cal_data.tail(5000))

In [ ]:
cal_data_cor=training_examples.copy()
cal_data_cor["target"]=training_targets["median_house_value"]
cal_data.corr()

In [ ]:
plt.figure(figsize=(13,8))

ax =plt.subplot(1,2,1)
ax.set_title("Validation")


ax.set_autoscaley_on(False)
ax.set_ylim([32,43])
ax.set_autoscalex_on(False)
ax.set_xlim([-126,-112])

plt.scatter(
    validation_examples["longitude"],
    validation_examples["latitude"],
    cmap="coolwarm",
    c=validation_targets["median_house_value"] / training_targets["median_house_value"].max()
    )


ax =plt.subplot(1,2,2)
ax.set_title("Training")


ax.set_autoscaley_on(False)
ax.set_ylim([32,43])
ax.set_autoscalex_on(False)
ax.set_xlim([-126,-112])

plt.scatter(
    training_examples["longitude"],
    training_examples["latitude"],
    cmap="coolwarm",
    c=training_targets["median_house_value"] / training_targets["median_house_value"].max()
    )

In [ ]:
def my_input_fn(
    features, 
    targets, 
    batch_size=1, 
    shuffle=True, 
    num_epochs=None):
    
        """ Parameters: 
        features: pd dataframe of feature
        targets: pd dataframe of targets
        batch_size: size of batch
        shuffle shuffle data or not 
        num_epochs:  number of runs 
        
        Returns:
        Tuple of (features, lables) for next data batch
        """
        
        # Convert df into dict of np.array.
        features = { 
                    key:np.array(value) 
                    for key, value in dict(features).items() 
                   }
        
        #Contruct a dataset, and configure batching/repaeating.
        ds= Dataset.from_tensor_slices((features, targets))
        ds= ds.batch(batch_size).repeat(num_epochs)
        
        # Shuffle if ordered
        if shuffle:
            ds =ds.shuffle(10000)
            
        # Return next batch of the data
        features, labels =ds.make_one_shot_iterator().get_next()
        
        return features, labels

In [ ]:
def train_nn_model(
    learning_rate, 
    steps, 
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
    
    # How often you wnat updates:    
    periods =10 
    steps_per_period = steps/periods
    
    """ Parameters:
    learning_rate: how big should our step should be 
    steps: how many times should we process batch
    batch_size: sample size 
    input_feature: what should be the input 
    """
    
    #Configure regressor:
    my_optimizer= tf.train.AdamOptimizer(learning_rate= learning_rate)
    my_optimizer= tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
    dnn_regressor = tf.estimator.DNNRegressor(
        feature_columns=construct_feature_columns(training_examples),
        hidden_units=hidden_units,
        optimizer=my_optimizer
        )
    
    # Creating input pfunction 
    
    training_input_fn = lambda: my_input_fn(
        training_examples,
        training_targets["median_house_value"],
        batch_size=batch_size)
    
    predict_training_input_fn = lambda: my_input_fn(
        training_examples,
        training_targets["median_house_value"],
        num_epochs=1,
        shuffle=False)
    
    predict_validation_input_fn = lambda: my_input_fn(
        validation_examples,
        validation_targets["median_house_value"],
        num_epochs=1,
        shuffle=False)
        

    # Trainign model from below code
    print("Training started:")
    print("RMSE on models are:")
    training_rmses=[]
    validation_rmses=[]
    
    
    for  period in  range(0, periods):
        dnn_regressor.train(
            input_fn = training_input_fn,
            steps = steps_per_period
            
        )
        
    
        # Take a break and and compute some predictions 
    
        training_predictions =dnn_regressor.predict(input_fn=predict_training_input_fn)
        training_predictions = np.array([
                            item["predictions"][0]
                            for item in training_predictions 
                           ])
    
        validation_predictions =dnn_regressor.predict(input_fn=predict_validation_input_fn)
        validation_predictions = np.array([
                            item["predictions"][0]
                            for item in validation_predictions 
                           ])
    
    
        # Compute loss
        training_mse=metrics.mean_squared_error(training_predictions, training_targets)
        training_rmse=math.sqrt( training_mse )
        
        validation_mse=metrics.mean_squared_error(validation_predictions, validation_targets)
        validation_rmse=math.sqrt( validation_mse )
        
        
        #Print current loss
        print("period %02d: %0.2f",period, training_rmse)
        
        #Save rmse
        training_rmses.append(training_rmse)
        validation_rmses.append(validation_rmse)
       
    print("Model training finished.")
    
    # Output graph of loss metrics over periods. 
    plt.ylabel('RMSE')
    plt.xlabel("Periods")
    plt.title("RMSE vs Periods")
    plt.tight_layout()
    plt.plot(training_rmses, label="training")
    plt.plot(validation_rmses, label="validation")
    plt.legend()
  

    return dnn_regressor

In [ ]:
cal_data.head()

In [ ]:
# Various function for Scaling
def linear_scale(series):
    min_val= series.min()
    max_val= series.max()
    scale = (max_val-min_val) / 2
    return series.apply(lambda x: ((x-min_val)/scale) - 1.0)

def log_scale(series):
    return series.apply( lambda x:math.log(x+1.0))

def clip(series):
    return series.apply ( lambda x: (
        min(max(x, clip_to_min), clip_to_max))
        )

def z_score(series):
    mean=series.mean()
    std_dv=series.std()
    return series.apply(lambda x: (x-mean)/std_dv )
    
def normalize_scale(df,method):
    processed_features= pd.DataFrame()
    
    target_list=["median_house_value"]
    for col in df.columns:
        if col not in target_list: 
            processed_features[col] = method(df[col])
        else:
            pass
    return processed_features

def binary_treshold(series, threshold):
    return series.apply(lambda x:(1 if x >threshold else 0))

def clipper(series, threshold=20):
    return series.apply(lambda x:(x if x < threshold else threshold))

normal_cal_data= normalize_scale(preprocess_features(cal_data), clipper)
normal_training_examples   = normal_cal_data.head(12000)
normal_validation_examples = normal_cal_data.tail(5000)
normal_cal_data

_=normal_training_examples.hist(bins=20, figsize=(20,20), xlabelsize=10 )


In [ ]:
dnn_regressor= train_nn_model(
    learning_rate=1,
    steps=2000,
    batch_size=100,
    hidden_units=[10,2],
    training_examples=normal_training_examples,
    training_targets=training_targets,
    validation_examples=normal_validation_examples,
    validation_targets=validation_targets
    )
 

In [ ]:
in_data=preprocess_features(cal_data).head(1)
out_data=preprocess_targets(cal_data).head(1)
out_data

In [ ]:
predict_training_input_fn = lambda: my_input_fn(
        in_data,
        out_data["median_house_value"],
        num_epochs=1,
        shuffle=False)

In [ ]:
ds_predict_tf = dnn_regressor.predict(input_fn = predict_training_input_fn)

list(ds_predict_tf)

In [ ]:
# plt.scatter(training_examples["median_income"], training_targets["median_house_value"])
#plt.scatter(training_examples["longitude"], training_targets["median_house_value"])



In [ ]:
# https://stackoverflow.com/questions/25119524/pandas-conditional-rolling-count
    
    
#Set up plotting
#     plt.figure(figsize=(15,6))
#     plt.subplot(1,2,1)
#     plt.title(" Learnd line period")
#     plt.ylabel(my_label)
#     plt.xlabel(my_feature)
    
#     sample=df.sample(n=300)
#     plt.scatter(sample[my_feature], sample[my_label])
#     colors = [cm.coolwarm(x) 
#              for x in np.linspace(-1, 1, periods)
#              ]
    
    

#         # PLot weights and bias using graph 
#         y_extents=np.array([  0, sample[my_label].max()   ])

#         weight = linear_regressor.get_variable_value('linear/linear_model/%s/weights' % input_feature)[0]
#         bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

#         x_extents = (y_extents - bias) / weight
#         x_extents = np.maximum(np.minimum(x_extents,
#                                       sample[my_feature].max()),
#                            sample[my_feature].min())
#         y_extents = weight * x_extents + bias
#         plt.plot(x_extents, y_extents, color=colors[period]) 


def binner(df, col, kitni_bar_katu):
    max_of_col=df[col].max()
    min_of_col=df[col].min()
    range_of_col= max_of_col -min_of_col
    
    print("MAX:%0.2f" % (max_of_col))
    print("MIN:%0.2f" % (min_of_col))

    long_bin=[]
    for i in range(0, 12):
        long_bin.append(min_long)
        min_long +=1

pd.get_dummies(training_examples["longitude"])
pd.qcut(training_examples["longitude"], 10, labels=False)

# g=tf.one_hot(indices=training_examples["longitude"], depth=10,on_value=1, off_value=0 )
# g